In [ ]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
from diets import load_model
import coralme
pandas.set_option('display.max_colwidth', None)

In [ ]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

### Set organisms

In [ ]:
# !bash report.sh
organisms = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
# done = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = set(i.split(".csv")[0] for i in os.listdir("./fluxes_reconstruction/me/") if "checkpoint" not in i)

In [ ]:
run_for = list(done)
len(run_for)

### Load

In [ ]:
from tqdm import tqdm
conditions = ["base"]
def load_data(path,org):
    try:
        df = pandas.read_csv(path,index_col=0)[["fluxes"]]
    except:
        df = pandas.DataFrame(columns=["fluxes"])
    df.columns = [org]
    return df

def load_growth(folder):
    growth = {i:{} for i in conditions}
    for org in tqdm(organisms):
        for c in conditions:
            tmp = load_data("./{}/fluxes/{}/{}.csv".format(folder,c,org),org)
            if tmp.empty:
                val = 0
            else:
                val = tmp[org]["biomass_dilution"]
            growth[c][org] = val
    return pandas.DataFrame.from_dict(growth)

In [ ]:
HFD = load_growth("cases-HFD")

In [ ]:
WD = load_growth("cases")

In [ ]:
df = WD.join(HFD,lsuffix="WD",rsuffix="HFD")

In [ ]:
df.sort_values("baseHFD")

### Classify

In [ ]:
growth = df[["baseWD"]]
growth.columns = ["base"]
growth = growth[growth["base"]>0]

In [ ]:
taxonomy = pandas.read_csv("./taxonomy.txt",index_col=0,sep='\t')
taxa = "Phylum"

In [ ]:
growth["taxa"] = taxonomy[taxa]
mean = growth.groupby("taxa").mean().sort_values("base").fillna(0.)
std = growth.groupby("taxa").std().sort_values("base").fillna(0.)

fig,ax = plt.subplots(1,1)
mean.plot.bar(ax=ax)
ax.errorbar(x=mean.index,y=mean["base"][mean.index],yerr=std["base"][mean.index],color='k')

In [ ]:
wd = mean

### Classify

In [ ]:
growth = df[["baseHFD"]]
growth.columns = ["base"]
growth = growth[growth["base"]>0]

In [ ]:
taxonomy = pandas.read_csv("./taxonomy.txt",index_col=0,sep='\t')
taxa = "Phylum"

In [ ]:
growth["taxa"] = taxonomy[taxa]
mean = growth.groupby("taxa").mean().sort_values("base").fillna(0.)
std = growth.groupby("taxa").std().sort_values("base").fillna(0.)

fig,ax = plt.subplots(1,1)
mean.plot.bar(ax=ax)
ax.errorbar(x=mean.index,y=mean["base"][mean.index],yerr=std["base"][mean.index],color='k')

In [ ]:
hfd = mean

In [ ]:
tmp = pandas.concat([wd.rename(columns={"base":"WD"}),hfd.rename(columns={"base":"HFD"})],axis=1)

In [ ]:
tmp.plot.bar()